# Multiple Peaks

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as sk_gp
import GPy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # prettify matplotlib

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui.jupyter as tg

In [ ]:
# make deterministic
np.random.seed(42)

In [ ]:
def RBF(x, c, beta):
    return np.exp(-beta * (x-c)**2)
f = lambda x: 2 * RBF(x, 5, 1e-3) + 1 * RBF(x, -3, 0.1) + 2 * RBF(x, 10, 10)

xmin, xmax = -10, 20
xs = np.linspace(xmin, xmax, num=1000)

ys = f(xs)
best_y = np.max(ys)
best_x = xs[np.argmin(ys)]

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xs, ys)
plt.show()

In [ ]:
op = tb.Optimiser(f, 'max', [('x', xmin, xmax)], pre_phase_trials=5, settings_preset='default')

#op.acquisition = tm.UCB(beta=lambda trial_num: 5*np.log(trial_num-2)-4)
#tp.plot_acquisition_parameter_function(op.acquisition.beta, 5, 30)

op.surrogate = tm.SciKitGPSurrogate(model_params=dict(
    kernel = 1.0 * sk_gp.kernels.RBF(length_scale_bounds=(1e-5, 1)) + sk_gp.kernels.WhiteKernel(),
), training_iterations=4)
rec = tb.Recorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=30)


In [ ]:
tp.plot_error(rec, true_best=best_y);
tp.plot_overview(rec)

In [ ]:
tp.interactive_plot_trial_1D(rec, true_objective=f)